In [2]:
import numpy as np
import json
import os
import pandas as pd
import matplotlib.pyplot as plt

from utils import load_env_file, set_mpl_configs
from utils import leave_percentile, distribution_analysis

load_env_file()
set_mpl_configs()

DATA_DIR = os.getenv('DATA_DIR')
print('DATA_DIR: {}'.format(DATA_DIR))

load env file
  root dir:
    /home/k/Repo/IBD-EDA
  current system:
    Linux
  load .env.linux
  loaded data dir:
    /home/k/Nutstore Files/毕设-EHR/DB
done.
set matplotlib configs
  font family:
    ['Times New Roman']
done.
DATA_DIR: /home/k/Nutstore Files/毕设-EHR/DB


In [3]:
with open('../data/ibd_demo.json', 'r') as f:
    data = json.loads(f.read())
    
both_ibd_patients: list = data['both_ibd']
only_uc_patiens: list = data['only_uc']
only_cd_patients: list = data['only_cd']


df = pd.read_csv(os.path.join(DATA_DIR, 'complication', 'Complications_Patients.csv'))
df_demography = df.groupby('subject_id').agg({
    'gender': 'first',
    'anchor_age': 'first',
    'anchor_year_group': 'first',
    'dod': 'first',
})

# df_demography = df_demography.loc[only_cd_patients, :]

In [4]:
def get_suspect_icd_list(threshold=300) -> list:
    stats = df.groupby(['subject_id', 'icd_code']).agg({
        'icd_code': ['count'],
    })
    stats.columns = ['count']
    stats = stats.reset_index()
    stats = stats.groupby('icd_code').agg({
        'subject_id': ['nunique']
    })
    stats.columns = ['count']
    return [_ for _ in stats[stats['count'] > threshold].index.tolist() if _ not in ['5550',
                                                                                     '5551',
                                                                                     '5552',
                                                                                     '5559',
                                                                                     '5560',
                                                                                     '5561',
                                                                                     '5562',
                                                                                     '5563',
                                                                                     '5564',
                                                                                     '5565',
                                                                                     '5566',
                                                                                     '5568',
                                                                                     '5569']]

In [5]:
# X
suspect_icd_list = get_suspect_icd_list(400)
# suspect_icd_list = both_ibd_patients
X = np.zeros((len(df_demography.index), 2 + len(suspect_icd_list)))

'''
0: gender | 1: age_group | 2: icd_2724 | 4: icd_27651 | 5: icd_2859 | 6: icd_30000 | 7: icd_311 | 8: icd_4019 | 9: icd_53081 | 10: icd_5849 | 11: icd_V1582
'''


for i in range(len(df_demography.index)):
    subject_id = df_demography.index[i]
    tmp_df = df[df['subject_id'] == subject_id]
    
    X[i, 0] = 1 if df_demography.loc[subject_id, 'gender'] == 'M' else 0
    X[i, 1] = 1 if int(df_demography.loc[subject_id, 'anchor_age']) >= 60 else 0
    # X[i, 2] = 1 if df_demography.loc[subject_id, 'anchor_year_group'] == 'Year 4' else 0
    
    for j in range(len(suspect_icd_list)):
        X[i, 2 + j] = 1 if (tmp_df['icd_code'] == suspect_icd_list[j]).any() else 0

X.shape

(2417, 11)

In [6]:
# y
labelDod = lambda x: 1 if type(x) == str else 0

y = df_demography['dod'].apply(labelDod).values
print('  dead nums: {} live nums: {}'.format(y[y == 1].shape, y[y == 0].shape))

  dead nums: (380,) live nums: (2037,)


In [7]:
# Save files
np.savetxt('../r scripts/data/X.csv', X, delimiter=',', header=','.join(['gender', 'age'] + suspect_icd_list), comments='')
np.savetxt('../r scripts/data/y.csv', y, delimiter=',', header='dod', comments='')

In [8]:
suspect_icd_list

['2724', '27651', '2859', '30000', '311', '4019', '53081', '5849', 'V1582']

# Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score

In [ ]:
[0, 1, 2] + get_suspect_icd_list(400)

In [ ]:
X = X[:, [1, 3, 5, 7, -2, -1]]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)

print('--> Logistic Regression')
print('  accuracy:\n    {:.4f}'.format(accuracy))
print('  precision:\n    {:.4f}'.format(precision))
print('  recall:\n    {:.4f}'.format(recall))
print('  f1:\n    {:.4f}'.format(f1))


In [ ]:
def logistic_regression(X: np.ndarray, y: np.ndarray, random_state: int = 42, test_size: float = 0.2) -> LogisticRegression:
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=random_state)

    model = LogisticRegression()
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print('--> Logistic Regression')
    print('  accuracy:\n    {:.4f}'.format(accuracy))
    print('  precision:\n    {:.4f}'.format(precision))
    print('  recall:\n    {:.4f}'.format(recall))
    print('  f1:\n    {:.4f}'.format(f1))
    
    return model

In [ ]:
model = logistic_regression(X, y)

In [ ]:
model = logistic_regression(X[:, [1, 3, 5, 7, 10, 11]], y, test_size=0.2, random_state=45)

In [ ]:
stats = df.groupby(['subject_id', 'icd_code']).agg({
    'icd_code': ['count'],
})
stats.columns = ['count']
stats = stats.reset_index()
stats = stats.groupby('icd_code').agg({
    'subject_id': ['nunique']
})
stats.columns = ['count']

stats[stats.index == '2761'], stats[stats.index == '2762'], stats[stats.index == '2768']

In [ ]:
get_suspect_icd_list(400)

In [25]:
380 / 2417, 14 / 85, 199 / 1365, 167 / 1137

(0.15721969383533305,
 0.16470588235294117,
 0.1457875457875458,
 0.1468777484608619)